In [63]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import matplotlib
import scipy
import requests
import os
# Silence Warnings
warnings.filterwarnings("ignore")


In [ ]:
locations = []
summer_years = range(1948, 2025, 4)
for sy in summer_years:
    url = f"https://en.wikipedia.org/wiki/{sy}_Summer_Olympics_medal_table"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(url, headers=headers)

    tables = pd.read_html(response.text)
    col = tables[0].columns[0]
    location_idx = tables[0][col].str.contains("Location").idxmax()
    location = tables[0].iloc[location_idx, 1]
    locations.append(("summer", sy, location))

    for table in tables:
        if "Gold" in table.columns and "Silver" in table.columns and "Bronze" in table.columns:
            medal_table = table
            break
    medal_table.to_csv(f"tables/summer_{sy}.csv", index=False)

for a,b in [(1948, 1993), (1994, 2027)]:
    winter_years = range(a, b, 4)
    for wy in winter_years:
        url = f"https://en.wikipedia.org/wiki/{wy}_Winter_Olympics_medal_table"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        response = requests.get(url, headers=headers)

        tables = pd.read_html(response.text)
        col = tables[0].columns[0]
        location_idx = tables[0][col].str.contains("Location").idxmax()
        location = tables[0].iloc[location_idx, 1]
        locations.append(("winter", wy, location))
        for table in tables:
            if "Gold" in table.columns and "Silver" in table.columns and "Bronze" in table.columns:
                medal_table = table
                break

        medal_table.to_csv(f"tables/winter_{wy}.csv", index=False)


In [107]:
# Compile all summer and winter CSVs
all_summer = pd.concat([pd.read_csv(f"tables/summer_{sy}.csv").assign(CY=sy) for sy in summer_years], ignore_index=True)
all_winter = pd.concat([pd.read_csv(f"tables/winter_{wy}.csv").assign(CY=wy) for wy in range(1948,2027) if os.path.exists(f"tables/winter_{wy}.csv")], ignore_index=True)

all_summer["NOC"] = all_summer["NOC"].combine_first(all_summer.pop("Nation")).str.replace(r'[^a-zA-Z ]', '', regex=True)
all_winter["NOC"] = all_winter["NOC"].combine_first(all_winter.pop("Nation")).str.replace(r'[^a-zA-Z ]', '', regex=True)

all_summer = all_summer[~all_summer["NOC"].str.contains("Total", case=False, na=False)]
all_winter = all_winter[~all_winter["NOC"].str.contains("Total", case=False, na=False)]

all_summer.to_csv("tables/summer_all.csv", index=False)
all_winter.to_csv("tables/winter_all.csv", index=False)

In [109]:
locations = pd.DataFrame(locations, columns=["Season", "Year", "Location"])
locations[["City", "Country"]] = locations["Location"].str.split(", ", expand=True)

In [138]:
summer = locations[locations["Season"] == "summer"]
years = summer["Year"].tolist()
hosts = {year: summer[summer["Year"] == year]["Country"].values[0] for year in years}
all_summer["IsHost"] = all_summer.apply(lambda row: hosts.get(row["CY"], "") == row["NOC"], axis=1)
all_summer = all_summer.sort_values(["NOC", "CY"]).reset_index(drop=True)

winter = locations[locations["Season"] == "winter"]
years = winter["Year"].tolist()
hosts = {year: winter[winter["Year"] == year]["Country"].values[0] for year in years}
all_winter["IsHost"] = all_winter.apply(lambda row: hosts.get(row["CY"], "") == row["NOC"], axis=1)
all_winter = all_winter.sort_values(["NOC", "CY"]).reset_index(drop=True)


In [148]:

def extract_host_window(df, x_window=1):
    # Find all host instances
    hosts_list = df[df['IsHost'] == True].copy()
    
    results = []
    
    for _, row in hosts_list.iterrows():
        country = row['NOC']
        host_year = row['CY']
        
        # Filter for just this country's history
        country_df = df[df['NOC'] == country].sort_values('CY').reset_index(drop=True)
        
        # Find the index of the host year in the country's timeline
        host_idx = country_df[country_df['CY'] == host_year].index[0]
        
        # Calculate start and end indices for the window
        start_idx = max(0, host_idx - x_window)
        end_idx = min(len(country_df), host_idx + x_window + 1)
        
        window_df = country_df.iloc[start_idx:end_idx].copy()
        
        # Relative positioning for the plot/table
        window_df['Relative_Year'] = window_df['CY'].apply(lambda y: f"{y - host_year:+d}" if y != host_year else "Host Year")
        window_df['Host_Country'] = country
        window_df['Original_Host_Year'] = host_year

        window_df = window_df[~window_df['Relative_Year'].str.contains("12")]
        window_df = window_df[~window_df['Relative_Year'].str.contains("16")]
        
        results.append(window_df)
        
    return pd.concat(results).reset_index(drop=True)

# Extract with window x=1 (1 before, 1 after)
df_window_2_summmer = extract_host_window(all_summer, x_window=2)

# Extract with window x=2 (2 before, 2 after)
df_window_2_winter = extract_host_window(all_winter, x_window=2)

df_window_2_summmer.sort_values(["Original_Host_Year", "Relative_Year"], ascending=[False, True]).to_csv("tables/summer_host_windows.csv", index=False)
df_window_2_winter.sort_values(["Original_Host_Year", "Relative_Year"], ascending=[False, True]).to_csv("tables/winter_host_windows.csv", index=False)



In [133]:
df_window_2

,Rank,Gold,Silver,Bronze,Total,CY,NOC,IsHost,Relative_Year,Host_Country,Original_Host_Year,Relative_Offset,Game_Label
0,14,2,6,5,13,1948,Australia,False,-8,Australia,1956,-8,Australia 1956
1,9,6,2,3,11,1952,Australia,False,-4,Australia,1956,-4,Australia 1956
2,3,13,8,14,35,1956,Australia,True,Host Year,Australia,1956,0,Australia 1956
3,5,8,8,6,22,1960,Australia,False,+4,Australia,1956,4,Australia 1956
4,8,6,2,10,18,1964,Australia,False,+8,Australia,1956,8,Australia 1956
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,1,36,39,26,101,2004,United States,False,+8,United States,1996,8,United States 1996
88,8,5,11,10,26,1968,West Germany,False,-4,West Germany,1972,-4,West Germany 1972
89,4,13,11,16,40,1972,West Germany,True,Host Year,West Germany,1972,0,West Germany 1972
90,4,10,12,17,39,1976,West Germany,False,+4,West Germany,1972,4,West Germany 1972
